<a href="https://colab.research.google.com/github/4inigriv/Segmenta-oFiveShots/blob/main/Segmenta%C3%A7aoComBase5exemplos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import numpy as np
import sklearn
import skimage
import os
import csv
import pickle
from skimage.feature import match_template
from scipy.signal import find_peaks
import argparse
import soundfile as sf
from scipy.signal import istft
from IPython.display import Audio

In [ ]:
!pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import soundfile as sf
import numpy as np

# Caminhos completos para os dois arquivos de áudio
caminho_arquivo_audio1 = '/content/drive/MyDrive/Cobra/SonsPassaros/AracuaPantanal -  Ortalis canicollis/XC351098-aracuã-do-pantanal-Ortalis-canicollis-pantanalensis-_ssp-assumed-from-location__1.wav' #audio fixo
caminho_arquivo_variosPassaros = '/content/drive/MyDrive/Cobra/SonsPassaros/CincoAves/GralhaAracuaChororoInambuQueroquero.wav' #juntar manualmente os audios
# Carregar os áudios
audio1, sr1 = sf.read(caminho_arquivo_audio1)
audio2, sr2 = sf.read(caminho_arquivo_variosPassaros)


In [ ]:
#concatenar AUDIOS
import wave

infiles = [caminho_arquivo_audio1, caminho_arquivo_variosPassaros]
outfile = "/content/drive/MyDrive/output_NewAudio.wav"

data= []
for infile in infiles:
    w = wave.open(infile, 'rb')
    data.append( [w.getparams(), w.readframes(w.getnframes())] )
    w.close()

output = wave.open(outfile, 'wb')
output.setparams(data[0][0])
for i in range(len(data)):
    output.writeframes(data[i][1])
output.close()

In [ ]:
# Carregar o áudio
y, sr = librosa.load(outfile)

# duração do áudio em segundos
duracao = librosa.get_duration(y=y, sr=sr)

print("A duração do áudio é:", duracao, "segundos")

A duração do áudio é: 69.94013605442177 segundos


In [ ]:
#Calcula a transformada de fourier
forma_de_onda, sr = librosa.load(outfile, sr = None)
nfft=int(sr/10)
hop_len = int(nfft/4)
stft = np.abs(librosa.stft(forma_de_onda, n_fft=nfft, hop_length=hop_len, window='hann', pad_mode='reflect'))

In [ ]:
#REDUÇÃO DE RUÍDO
#lembrar de utilizar wavelet

stft_mediana = np.median(stft, axis=-1, keepdims=True)
stft_tempo_mediana = np.median(stft, axis=0, keepdims=True)
norm_stft = stft - stft_mediana
norm_stft = norm_stft - stft_tempo_mediana

audio = librosa.istft(norm_stft) #função q facilita transformar a matriz em onda p ouvir o audio

In [ ]:
#eventos de interesse dentro do seu áudio (escritos manualmente)
eventos = [
    (1.10, 1.13),
    (1.40, 1.42),
    (1.50, 1.51),
    (1.57, 1.58)
]
#arrumar os seg
#verificar como ele recebe os intervalos (qual unidade)

In [ ]:
# Seção de STFT a ser usada para previsão; inicia após o tempo final do último evento do modelo
to_predict = norm_stft[:, int(np.ceil(float(eventos[-1][-2])*sr/hop_len + 1)):]

In [ ]:
#resultados das SIMILARIDADES encontradas dentro do audio

import numpy as np
import matplotlib.pyplot as plt

resultado = []
for evento in eventos:
    tempo_inicial = float(evento[1])
    tempo_final = float(evento[-1])
    evento_stft = norm_stft[2:-2, int(np.floor(tempo_inicial * sr / hop_len + 1)):int(np.ceil(tempo_final * sr / hop_len + 1))]

    # Verificar se o tamanho do evento_stft é menor do que o tamanho de to_predict
    if evento_stft.shape[1] <= to_predict.shape[1]:
        resultado.append(match_template(to_predict, evento_stft))
    else:
        print("O evento_stft é maior do que to_predict e não pode ser usado como template.") #perdendo o final do audio pq tem que ser menor >.<

#matriz de resultados dos eventos de interesse
print(resultado) #veja a quantidade de elementos, e oqnt esta retornando
print(len(resultado))

[array([[ 0.22375017,  0.41932052,  0.3808324 , ...,  0.07885331,
        -0.15688683,  0.32673785],
       [ 0.45849466,  0.6522824 ,  0.52015966, ...,  0.085518  ,
        -0.08981849,  0.28717333],
       [ 0.88216823,  0.936119  ,  0.8684959 , ...,  0.37753722,
         0.08224799,  0.24028832],
       [ 0.95058095,  0.9579958 ,  0.9519506 , ...,  0.72774655,
         0.24099612,  0.54531145],
       [ 0.8230526 ,  0.72042304,  0.9045713 , ...,  0.6921572 ,
         0.49594283,  0.5874418 ]], dtype=float32), array([[0.2097616 , 0.33299252, 0.26054928, ..., 0.20512721, 0.10427694,
        0.38280964],
       [0.5416382 , 0.57722527, 0.51463187, ..., 0.2525646 , 0.19007196,
        0.26343048],
       [0.48624626, 0.37778744, 0.4791896 , ..., 0.4989722 , 0.20117004,
        0.4194496 ],
       [0.31451347, 0.29479596, 0.3131635 , ..., 0.4611933 , 0.47434762,
        0.5123004 ],
       [0.4613415 , 0.5956493 , 0.38360307, ..., 0.23002252, 0.19534566,
        0.26029548]], dtype=float

In [ ]:
#MR armazena o resultado MAXIMO de comparaçao entre os eventos
from skimage.feature import match_template
mr = []
for i in range(len(eventos)):
    evento = eventos[i]
    tempo_inicial = float(evento[1])
    tempo_final = float(evento[-1])
    evento_stft = norm_stft[2:-2, int(np.floor(tempo_inicial * sr / hop_len + 1)):int(np.ceil(tempo_final * sr / hop_len + 1))]

    r = []
    for j in range(len(eventos)):
        if j != i:
            evento_interno = eventos[j]
            tempo_inicial_interno = float(evento_interno[0])
            tempo_final_interno = float(evento_interno[1])
            evento_interno_stft = norm_stft[2:-2, int(np.floor(tempo_inicial_interno * sr / hop_len + 1)):int(np.ceil(tempo_final_interno * sr / hop_len + 1))]

            if evento_interno_stft.shape[1] >= evento_stft.shape[1]:
                r.append(np.max(match_template(evento_interno_stft, evento_stft)))
    if r:
        mr.append(np.max(r))
print("medida numérica de quão semelhantes são os eventos comparados na lista eventos:",mr)

medida numérica de quão semelhantes são os eventos comparados na lista eventos: [0.91546255, 0.63361776, 0.9297595, 0.9297586]


In [ ]:
#pega o maior valor de mr para achar o limiar
limiar = np.max(mr)
print("o numero MAIS semalhante que mais se aproxima de 1:", limiar)

o numero MAIS semalhante que mais se aproxima de 1: 0.9297595


In [ ]:
from scipy.signal import find_peaks #Encontre picos dentro de um sinal

# Seleção de picos no vetor de resultados de match_template, transformando picos em 1 e o resto em 0
resultado_binario = []
rmax = []
for i in range(len(resultado)):
    tamanho_evento = int(np.ceil(np.floor(float(eventos[i][1]) * sr / hop_len + 1))) - int(np.floor(float(eventos[i][-1]) * sr / hop_len))
    rmax = np.zeros(resultado[i].shape[1])
    picos, _ = find_peaks(np.max(resultado[i], axis=0), height=limiar, distance = tamanho_evento)
    rmax[picos] = 1
    resultado_binario.append(rmax)

print(resultado_binario)


[array([0., 1., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 1., ..., 0., 0., 0.])]


In [ ]:
to_write = []
# sobreposição das previsões do resultado_binario
for i in range(1, len(resultado_binario)):
    # Converter os tempos de início e fim para pontos flutuantes
    tempo_inicial = float(eventos[i][0])
    tempo_final = float(eventos[i][1])

    # Extrair o STFT do evento com base nos tempos de início e fim
    stft_evento = norm_stft[2:-2, int(np.floor(tempo_inicial * sr / hop_len + 1)):int(np.ceil(tempo_final * sr / hop_len + 1))]

    # Calcular o comprimento do evento em número de frames
    comprimento_evento = int(stft_evento.shape[1])

    # Calcular a quantidade de preenchimento necessário para a esquerda (lpad) e direita (rpad)
    lpad = int(np.floor(comprimento_evento / 2))
    rpad = int(comprimento_evento - lpad - 1)

    # Encontrar os índices onde as previsões binárias são iguais a 1
    indices = np.where(resultado_binario[i] == 1)[0]

    # Soma dos resultados
    resultado_binario[0] += resultado_binario[i]

resultado_final = resultado_binario[0]
resultado_final[resultado_final > 0] = 1

# transformando quadros de tempo em segundos nas previsões
startind = np.where(resultado_final[:-1] - resultado_final[1:] == -1)[0]
endind = np.where(resultado_final[:-1] - resultado_final[1:] == 1)[0]

# preenchendo a lista to_write com os resultados previstos
for i in range(len(startind)):
    to_write.append([audio, startind[i] * hop_len / sr, endind[i] * hop_len / sr])

In [ ]:
print(startind)
print(endind)

[   0    3    6    9   13   19   22   26   28   31   34  532  538  547
  562  572  578  583  598  631  637  642  654  661  664 1125 1137 1147
 1247 1256 1259 1264 1279 1290 1491 1498 1532 1534 1540 1542 1548 1716
 2290 2293 2297 2462 2472 2474 2477 2482 2486 2489 2494 2497 2500 2502
 2505 2520 2525 2531 2535 2541]
[   2    5    8   10   16   20   25   27   29   32   35  533  539  548
  563  573  582  584  599  633  638  643  655  662  665 1126 1138 1148
 1248 1257 1260 1265 1280 1291 1492 1499 1533 1535 1541 1543 1549 1717
 2291 2294 2298 2463 2473 2475 2478 2483 2487 2490 2495 2498 2501 2503
 2506 2521 2526 2532 2536 2542]


In [ ]:
# Imprimir os pares de índices de início e fim
print("Pares de índices de início e fim dos eventos:")
for start, end in zip(startind, endind):
    print("[{}, {}]".format(start, end))

Pares de índices de início e fim dos eventos:
[0, 2]
[3, 5]
[6, 8]
[9, 10]
[13, 16]
[19, 20]
[22, 25]
[26, 27]
[28, 29]
[31, 32]
[34, 35]
[532, 533]
[538, 539]
[547, 548]
[562, 563]
[572, 573]
[578, 582]
[583, 584]
[598, 599]
[631, 633]
[637, 638]
[642, 643]
[654, 655]
[661, 662]
[664, 665]
[1125, 1126]
[1137, 1138]
[1147, 1148]
[1247, 1248]
[1256, 1257]
[1259, 1260]
[1264, 1265]
[1279, 1280]
[1290, 1291]
[1491, 1492]
[1498, 1499]
[1532, 1533]
[1534, 1535]
[1540, 1541]
[1542, 1543]
[1548, 1549]
[1716, 1717]
[2290, 2291]
[2293, 2294]
[2297, 2298]
[2462, 2463]
[2472, 2473]
[2474, 2475]
[2477, 2478]
[2482, 2483]
[2486, 2487]
[2489, 2490]
[2494, 2495]
[2497, 2498]
[2500, 2501]
[2502, 2503]
[2505, 2506]
[2520, 2521]
[2525, 2526]
[2531, 2532]
[2535, 2536]
[2541, 2542]


In [ ]:
print(to_write)

[[array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.0, 0.049977324263038546], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.07496598639455783, 0.12494331065759637], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.14993197278911566, 0.19990929705215418], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.22489795918367347, 0.24988662131519274], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.32485260770975055, 0.39981859410430837], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
        0.0011301 ,  0.00132649], dtype=float32), 0.47478458049886624, 0.4997732426303855], [array([-0.00100884, -0.00093818, -0.00089488, ...,  0.00088215,
  

In [ ]:
os.remove(outfile)